# Keypoint Detection of a Tennis Court

Based on : https://github.com/yastrebksv/TennisCourtDetector?tab=readme-ov-file

## Dataset

The Dataset was taken from the GitHub repository and consists of 8841 images, which were separeted to train set (75%) and validation set (25%). Each image has 14 annotated points. The resolution of images is 1280×720. This dataset contains all court types (hard, clay and grass).



## Model Training

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
from torchvision import models, transforms
import json
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [3]:
class KeyPointsDataset(Dataset):
    def __init__(self, img_dir,data_file):
        self.img_dir = img_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose(
            [transforms.ToPILImage(),
            # Resize for faster training (TODO: ?)
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            # Normalize using the mean & std of ImageNet (since we use a pretrained
            # ResNet model, which was trained on ImageNet)
            transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])]
        )


    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        img_h, img_w = img.shape[0], img.shape[1]

        # cv2 reads in BGR, so we convert
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        # Get Keypoints as 1D array of floats
        img_kps = np.array(item['kps']).flatten()
        img_kps = img_kps.astype(np.float32)

        # Convert keypoint positions to new scale (since we rescaled the image)

        # x coordinates (stored at 0,2,4,...)
        img_kps[::2] *= 224.0 /img_w
        # y coordinates (stored at 1,3,5,...)
        img_kps[1::2] *= 224.0 /img_h

        return img, img_kps







In [4]:
train_dataset = KeyPointsDataset(img_dir = '../data/keypoint_detection/dataset/images', data_file = '../data/keypoint_detection/dataset/data_train.json')
val_dataset = KeyPointsDataset(img_dir = '../data/keypoint_detection/dataset/images', data_file = '../data/keypoint_detection/dataset/data_val.json')

train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 16, shuffle = True)

For the model, we will finetune a pretrained ResNet model.

In [5]:
model = models.resnet34(weights='IMAGENET1K_V1')

# Fine-tuning only the last fully connected layer to output 14 keypoints (each has
# a x & y coordinate, therefore in total 14*2 values)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)


model = model.to(device = device)



In [6]:
criterion = torch.nn.MSELoss() # as we have a regression task (estimating coordinates), we can use MSELoss
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

In [7]:
epochs = 20

for epoch in range(epochs):
    for i, (imgs, imgs_kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        imgs_kps = imgs_kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, imgs_kps) # MSE between predicted & actual keypoints
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch : {epoch} , Step : {i} , Loss : {loss.item()}")

Epoch : 0 , Step : 0 , Loss : 15046.0859375
Epoch : 0 , Step : 10 , Loss : 14092.5947265625
Epoch : 0 , Step : 10 , Loss : 14092.5947265625
Epoch : 0 , Step : 20 , Loss : 14613.1015625
Epoch : 0 , Step : 20 , Loss : 14613.1015625
Epoch : 0 , Step : 30 , Loss : 14655.1953125
Epoch : 0 , Step : 30 , Loss : 14655.1953125
Epoch : 0 , Step : 40 , Loss : 13896.6640625
Epoch : 0 , Step : 40 , Loss : 13896.6640625
Epoch : 0 , Step : 50 , Loss : 14536.724609375
Epoch : 0 , Step : 50 , Loss : 14536.724609375
Epoch : 0 , Step : 60 , Loss : 13504.267578125
Epoch : 0 , Step : 60 , Loss : 13504.267578125
Epoch : 0 , Step : 70 , Loss : 14916.3974609375
Epoch : 0 , Step : 70 , Loss : 14916.3974609375
Epoch : 0 , Step : 80 , Loss : 13683.982421875
Epoch : 0 , Step : 80 , Loss : 13683.982421875
Epoch : 0 , Step : 90 , Loss : 13204.4013671875
Epoch : 0 , Step : 90 , Loss : 13204.4013671875
Epoch : 0 , Step : 100 , Loss : 13258.9189453125
Epoch : 0 , Step : 100 , Loss : 13258.9189453125
Epoch : 0 , Step :

In [8]:
# Test score on validation set (i.e. we just use as a test set here )

model.eval()
total_loss = 0

for i, (imgs, imgs_kps) in enumerate(val_loader):
    imgs = imgs.to(device)
    imgs_kps = imgs_kps.to(device)

    outputs = model(imgs)
    loss = criterion(outputs, imgs_kps)

    total_loss += loss.item()

print(f"Validation Loss : {total_loss / len(val_loader)}")


Validation Loss : 9.2226063021653


In [9]:
torch.save(model.state_dict(), "keypoints_model.pth")